In [ ]:

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
import os
root_dir=os.getcwd()
src_path_train = "data/chest_xray/train"
src_path_test = "data/chest_xray/test"